# Importing Libraries

In [ ]:
from numpy import log as ln
import numpy as np
from scipy.stats import norm
import pandas as pd
from numpy.ma.core import log
from statistics import mean

In [ ]:
df = pd.read_csv('Data.csv') 
print(df)

             Date  Price_appl  Price_amzn  Price_appl.1
0    Mar 22, 2021       123.0      3110.0         670.0
1    Mar 23, 2021       123.0      3140.0         662.0
2    Mar 24, 2021       120.0      3090.0         630.0
3    Mar 25, 2021       121.0      3050.0         640.0
4    Mar 26, 2021       121.0      3050.0         619.0
..            ...         ...         ...           ...
269  Apr 13, 2022       170.0      3110.0        1020.0
270  Apr 14, 2022       165.0      3030.0         985.0
271  Apr 18, 2022       165.0      3060.0        1000.0
272  Apr 19, 2022       167.0      3160.0        1030.0
273  Apr 20, 2022       167.0      3080.0         977.0

[274 rows x 4 columns]


# Defining the function for Binomial Model and Black Scholes Model.

In [ ]:
def binomial_model(s0,k,T,u,d,n,r):
  dt = T/n
  p = ( 2.72**(r*dt) - d)/(u-d)
  div =  2.72**(r*dt)
  iterations = n + 1
  s_n = []
  for i in range(iterations):
    x = s0*(u**(n-i))*(d**i)
    if x-k > 0:
      s_n.append(x-k)
    else:
      s_n.append(0)
  u = []
  l = []
  for i in range(len(s_n)-1):
    l.append((p*s_n[i]+(1-p)*s_n[i+1])/div)
  u.append(l)
  for i in range(n-1):
    l = []
    for j in range(len(u[i])-1):
      l.append((p*u[i][j]+(1-p)*u[i][j+1])/div)
    u.append(l)
  '''
  print("r : ",r)
  print("r * dt", r*dt)
  print("e^r*dt : ",div)
  print("P : ",p)
  print("1-P : ",1-p)
  print("u : ",u)
  print("d : ",d)
  print("s0 : ",s0)
  print("st :",s_n)
  for i in range(len(u)):
    print(u[i])
  '''
  return u[len(u)-1]

def call_option_parity(s0,c0,k,r,T):
  p0 = c0 - s0 + k * (2.72**(-r*T))
  return p0

def black_scholes(s,k,r,T,sigma):
  x = sigma * np.sqrt(T)
  d1 = ((ln(s/k) + (r + (sigma**2)*0.5)*T)/x)
  d2 = d1 - x
  nd1 = norm.cdf(d1)
  nd2 = norm.cdf(d2)
  c = s * nd1 - k * 2.72**(-r*T) * nd2 
  '''
  print("d1 : ",d1)
  print("nd1 : ",nd1)
  print("d2 : ",d2)
  print("nd2 : ",nd2)
  '''
  return d1,d2,c

def implied_volatility(cm,s,k,r,T):
  e = 1
  tol = 1e-4
  c = 1
  sigma_implied = 0.001
  f = 0.001
  '''
  while e > tol:
    c = c + 1
    if c > 10000:
      print("Breaking the Loop since max count reached")
      break
    sigma_old = sigma_implied
    d1,d2,c0 = black_scholes(s,k,r,T,sigma_old)
    function_value = cm - c0
    diff = s * norm.cdf(d1) * np.sqrt(T)
    if diff:
      sigma_implied = sigma_old -function_value/diff
    e = abs((sigma_implied-sigma_old)/sigma_old) 
  '''
  #print(cm,s,k,r,T)
  x = []
  y = []
  for i in range(999):
    d1,d2,ci = black_scholes(s,k,r,T,sigma_implied)
    #print(ci)
    x.append(abs((cm-ci)/cm))
    y.append(sigma_implied)
    #print("For simga : ",sigma_implied," difference between market and blackscholes price is ",abs(cm-ci))
    sigma_implied +=f 
  index = x.index(min(x))
  return y[index]

def self_financing_strategy(s0,sts,cts,r,Time):
  strategy = []
  for i in range(len(sts)):
    Time[i]/=252
    #print(cts[i],s0,sts[i],r,Time[i])
    sigma = implied_volatility(cts[i],s0,sts[i],r,Time[i])
    x = sigma * np.sqrt(Time[i])
    print("Implied Volatility for week ",i+1," is ",sigma)
    d1 = ((ln(s0/sts[i]) + (r + (sigma**2)*0.5)*Time[i])/x)
    nd1 = norm.cdf(d1)
    strategy.append(nd1)
  return strategy

In [ ]:
def sigma_from_data(p):
  r = []
  l = len(p)
  for i in range(1,l):
    r.append(ln(float(p[i])/float(p[i-1])))
  print(r)
  m = mean(r)
  sum = 0
  for i in range(len(r)):
    sum += (r[i]-m)**2
  sum = sum/len(r)
  s = np.sqrt(sum)
  sigma = s/np.sqrt(k/252) 
  return s,sigma

def self_financing_strategy_sigma(s0,sts,cts,r,T,s):
  strategy = []
  for i in range(len(sts)):
    T[i]/=252
    #print(T[i],s)
    sigma = s/np.sqrt(T[i])
    x = sigma * np.sqrt(T[i])
    print("Implied Volatility for week ",i+1," is ",sigma)
    d1 = ((ln(s0/sts[i]) + (r + (sigma**2)*0.5)*T[i])/x)
    nd1 = norm.cdf(d1)
    strategy.append(nd1)
  return strategy

# Testing the Models for a test case:

In [ ]:
s0 = 100
T = 45/252
sigma = 0.35
r = 0.05
n = 500
k = 80
u = 2.72**(sigma * np.sqrt(T/n))
d = 1/u
print("Initial Parameters")
print("r : ",r)
print("u : ",u)
print("d : ",d)
print("k : ",k)
print("T : ",T)
print("s0 : ",s0)
print("n : ",n)

Initial Parameters
r :  0.05
u :  1.006640508827928
d :  0.9934032966389761
k :  80
T :  0.17857142857142858
s0 :  100
n :  500


In [ ]:
c1 = binomial_model(s0,k,T,u,d,n,r)[0]
p1 = call_option_parity(s0,c1,k,r,T)
d1,d2,c2= black_scholes(s0,k,r,T,sigma)
p2 = call_option_parity(s0,c2,k,r,T)
print("For Binomial Model with n = ",n)
print("c0 : ",c1)
print("p0 : ",p1)
print("For Black Scholes Model:")
print("c0 : ",c2)
print("p0 : ",p2)
print("As we can see the difference between them is : ",abs(c2-c1))

For Binomial Model with n =  500
c0 :  21.040408287845587
p0 :  0.32885454990966423
For Black Scholes Model:
c0 :  21.039971395137954
p0 :  0.3284176572020243
As we can see the difference between them is :  0.00043689270763280774


In [ ]:
sigma_i = implied_volatility(c1,s0,k,r,T)
print("Sigma Implied is Calculated as : ",sigma_i)

Sigma Implied is Calculated as :  0.35000000000000026


# For Analysis of three different stocks:
# 1. Apple Stock [link](https://www.investing.com/equities/apple-computer-inc-options)

In [ ]:
s0 = 167.23
T = 93/252
sigma = 0.3142
r = 0.05
n = 500
k = 165
u = 2.72**(sigma * np.sqrt(T/n))
d = 1/u
print("Initial Parameters for today")
print("r : ",r)
print("u : ",u)
print("d : ",d)
print("k : ",k)
print("T : ",T)
print("s0 : ",s0)
print("n : ",n)

Initial Parameters for today
r :  0.05
u :  1.0085781337261415
d :  0.9914948248040536
k :  165
T :  0.36904761904761907
s0 :  167.23
n :  500


In [ ]:
c1 = binomial_model(s0,k,T,u,d,n,r)[0]
p1 = call_option_parity(s0,c1,k,r,T)
d1,d2,c2= black_scholes(s0,k,r,T,sigma)
p2 = call_option_parity(s0,c2,k,r,T)
print("For Binomial Model with n = ",n)
print("c0 : ",c1)
print("p0 : ",p1)
print("For Black Scholes Model:")
print("c0 : ",c2)
print("p0 : ",p2)
print("As we can see the difference between them is : ",abs(c2-c1))

For Binomial Model with n =  500
c0 :  15.323247433620892
p0 :  10.074634382164163
For Black Scholes Model:
c0 :  15.313376173679927
p0 :  10.064763122223212
As we can see the difference between them is :  0.00987125994096516


In [ ]:
sigma_i = implied_volatility(c2,s0,k,r,T)
print("Sigma Implied is Calculated as : ",sigma_i)

Sigma Implied is Calculated as :  0.3140000000000002


In [ ]:
price = df.iloc[:-180,1]
s,sigma = sigma_from_data(price)
print(s*np.sqrt(252/1))

[0.0, -0.02469261259037152, 0.008298802814695064, 0.0, 0.0, -0.008298802814695066, 0.016529301951210506, 0.008163310639160835, 0.024097551579060524, 0.0, 0.015748356968139112, 0.015504186535965254, 0.022814677766171264, -0.01515180502060222, 0.022642476749759752, -0.015037877364540559, 0.022472855852058576, -0.00743497848751809, 0.0074349784875179905, -0.014925650216675706, 0.007490671729157659, -0.015037877364540559, 0.015037877364540502, 0.0074349784875179905, -0.00743497848751809, 0.0, -0.007490671729157626, -0.01515180502060222, 0.015151805020602246, -0.03831886430213666, 0.0, 0.015504186535965254, 0.0, -0.023347363996991177, -0.007905179507113261, -0.02409755157906053, 0.016129381929883717, 0.015873349156290163, -0.007905179507113261, -0.007968169649176845, 0.0, 0.015873349156290163, -0.01587334915629012, 0.015873349156290163, 0.0, 0.0, -0.01587334915629012, 0.0, -0.008032171697264267, 0.008032171697264253, -0.008032171697264267, 0.01600034134644112, 0.0, 0.007905179507113247, 0.0

In [ ]:
s0 = 167.23
sts = [165,165,165,165,165,165,165,165,165,165]
cts = [2.91,5.35,6.05,6.55,7.1,7.6,9.3,10.75,12.4,14.25]
Time = [1,8,12,17,20,24,28,48,67,93]
strategy = self_financing_strategy(s0,sts,cts,r,Time)
for i in range(len(strategy)):
  print("For week ",i+1," the strategy or delta t value is : ",strategy[i])

Implied Volatility for week  1  is  0.36400000000000027
Implied Volatility for week  2  is  0.33700000000000024
Implied Volatility for week  3  is  0.32000000000000023
Implied Volatility for week  4  is  0.2930000000000002
Implied Volatility for week  5  is  0.2970000000000002
Implied Volatility for week  6  is  0.2930000000000002
Implied Volatility for week  7  is  0.34600000000000025
Implied Volatility for week  8  is  0.3030000000000002
Implied Volatility for week  9  is  0.2960000000000002
Implied Volatility for week  10  is  0.2870000000000002
For week  1  the strategy or delta t value is :  0.7276043549615879
For week  2  the strategy or delta t value is :  0.6102747069137797
For week  3  the strategy or delta t value is :  0.6030538299370662
For week  4  the strategy or delta t value is :  0.6020968477746342
For week  5  the strategy or delta t value is :  0.5985939298734981
For week  6  the strategy or delta t value is :  0.597290868616928
For week  7  the strategy or delta t v

In [ ]:
s0 = 167.23
sts = [165,165,165,165,165,165,165,165,165,165]
Time = [1,8,12,17,20,24,28,48,67,93]
strategy = self_financing_strategy_sigma(s0,sts,cts,r,Time,s)
for i in range(len(strategy)):
  print("For week ",i+1," the strategy or delta t value is : ",strategy[i])

Implied Volatility for week  1  is  0.4322344719126786
Implied Volatility for week  2  is  0.1528179630760207
Implied Volatility for week  3  is  0.12477534435591038
Implied Volatility for week  4  is  0.10483225780759081
Implied Volatility for week  5  is  0.0966505661415473
Implied Volatility for week  6  is  0.08822949211895083
Implied Volatility for week  7  is  0.08168463719644861
Implied Volatility for week  8  is  0.06238767217795519
Implied Volatility for week  9  is  0.05280584526521296
Implied Volatility for week  10  is  0.04482062681707625
For week  1  the strategy or delta t value is :  0.696353921965916
For week  2  the strategy or delta t value is :  0.7139467035827519
For week  3  the strategy or delta t value is :  0.7237773312841909
For week  4  the strategy or delta t value is :  0.7358276853489824
For week  5  the strategy or delta t value is :  0.7429271442915077
For week  6  the strategy or delta t value is :  0.7522363815111255
For week  7  the strategy or delta 

# 2. Amazon Stock [link](https://www.investing.com/equities/amazon-com-inc-options)

In [ ]:
s0 = 3079.96
T = 93/252
sigma = 0.3571
r = 0.05
n = 500
k = 3070
u = 2.72**(sigma * np.sqrt(T/n))
d = 1/u
print("Initial Parameters")
print("r : ",r)
print("u : ",u)
print("d : ",d)
print("k : ",k)
print("T : ",T)
print("s0 : ",s0)
print("n : ",n)

Initial Parameters
r :  0.05
u :  1.0097550637663424
d :  0.9903391781666769
k :  3070
T :  0.36904761904761907
s0 :  3079.96
n :  500


In [ ]:
c1 = binomial_model(s0,k,T,u,d,n,r)[0]
p1 = call_option_parity(s0,c1,k,r,T)
d1,d2,c2= black_scholes(s0,k,r,T,sigma)
p2 = call_option_parity(s0,c2,k,r,T)
print("For Binomial Model with n = ",n)
print("c0 : ",c1)
print("p0 : ",p1)
print("For Black Scholes Model:")
print("c0 : ",c2)
print("p0 : ",p2)
print("As we can see the difference between them is : ",abs(c2-c1))

For Binomial Model with n =  500
c0 :  297.72783344145427
p0 :  231.6033360598044
For Black Scholes Model:
c0 :  297.550057095915
p0 :  231.4255597142651
As we can see the difference between them is :  0.17777634553925736


In [ ]:
sigma_i = implied_volatility(c2,s0,k,r,T)
print("Sigma Implied is Calculated as : ",sigma_i)

Sigma Implied is Calculated as :  0.35700000000000026


In [ ]:
price = df.iloc[:-180,2]
s,sigma = sigma_from_data(price)
print(s*np.sqrt(252/1))

[0.009600073729019137, -0.0160517090105079, -0.013029500290333796, 0.0, 0.009788006366162944, -0.006514681021193642, 0.009756174945364656, 0.022400936689166713, 0.021910109635744326, -0.003100777678248271, 0.018462062839735352, 0.0060790460763821925, 0.02099027589183583, 0.0029629651306570487, 0.005899722127188322, -0.020803127629763284, 0.014903405502574948, 0.005899722127188322, -0.008862687257845317, -0.011940440371917863, 0.008968669982760316, -0.01499278458614128, 0.009022617599637552, 0.020741484306816372, 0.0029282597790883597, 0.011628037995119214, 0.0028860048891348514, 0.0, -0.023324672566408893, -0.02388173200338736, -0.012158204479809519, 0.012158204479809583, -0.006060624611690958, -0.030866647980527268, 0.009360442759563672, -0.02197890671877523, 0.0031695747612790395, 0.018809331957496293, 0.015408625352845061, -0.012307847674596814, 0.0, 0.006172859107081016, -0.015504186535965199, 0.01242251999855711, 0.006153865574378286, 0.003062789530545731, -0.012307847674596814, -

In [ ]:
s0 = 3079.96
sts = [3070,3070,3070,3070,3070,3070,3070,3070,3070,3070]
cts = [37,102,83,127,135,172,175,166,230,270.95]
Time = [1,8,12,17,20,24,28,48,67,93]
strategy = self_financing_strategy(s0,sts,cts,r,Time)
for i in range(len(strategy)):
  print("For week ",i+1," the strategy or delta t value is : ",strategy[i])

Implied Volatility for week  1  is  0.4070000000000003
Implied Volatility for week  2  is  0.43200000000000033
Implied Volatility for week  3  is  0.2770000000000002
Implied Volatility for week  4  is  0.36700000000000027
Implied Volatility for week  5  is  0.35900000000000026
Implied Volatility for week  6  is  0.4220000000000003
Implied Volatility for week  7  is  0.3950000000000003
Implied Volatility for week  8  is  0.2730000000000002
Implied Volatility for week  9  is  0.32500000000000023
Implied Volatility for week  10  is  0.32100000000000023
For week  1  the strategy or delta t value is :  0.5583917648712073
For week  2  the strategy or delta t value is :  0.5402997497044508
For week  3  the strategy or delta t value is :  0.5490249207261282
For week  4  the strategy or delta t value is :  0.5465805551753147
For week  5  the strategy or delta t value is :  0.548483690543196
For week  6  the strategy or delta t value is :  0.5503525658632864
For week  7  the strategy or delta t 

In [ ]:
s0 = 3079.96
sts = [3070,3070,3070,3070,3070,3070,3070,3070,3070,3070]
Time = [1,8,12,17,20,24,28,48,67,93]
strategy = self_financing_strategy_sigma(s0,sts,cts,r,Time,s)
for i in range(len(strategy)):
  print("For week ",i+1," the strategy or delta t value is : ",strategy[i])

Implied Volatility for week  1  is  0.4322344719126786
Implied Volatility for week  2  is  0.1528179630760207
Implied Volatility for week  3  is  0.12477534435591038
Implied Volatility for week  4  is  0.10483225780759081
Implied Volatility for week  5  is  0.0966505661415473
Implied Volatility for week  6  is  0.08822949211895083
Implied Volatility for week  7  is  0.08168463719644861
Implied Volatility for week  8  is  0.06238767217795519
Implied Volatility for week  9  is  0.05280584526521296
Implied Volatility for week  10  is  0.04482062681707625
For week  1  the strategy or delta t value is :  0.5556148562818539
For week  2  the strategy or delta t value is :  0.5756861158725061
For week  3  the strategy or delta t value is :  0.5870713119082589
For week  4  the strategy or delta t value is :  0.6011993978657266
For week  5  the strategy or delta t value is :  0.6096142972915114
For week  6  the strategy or delta t value is :  0.6207541312377063
For week  7  the strategy or delta

# 3. Tesla Stock [link](https://www.investing.com/equities/tesla-motors-options)

In [ ]:
s0 = 977.2
T = 93/252
sigma = 0.5986
r = 0.05
n = 500
k = 975
u = 2.72**(sigma * np.sqrt(T/n))
d = 1/u
print("Initial Parameters")
print("r : ",r)
print("u : ",u)
print("d : ",d)
print("k : ",k)
print("T : ",T)
print("s0 : ",s0)
print("n : ",n)

Initial Parameters
r :  0.05
u :  1.016406112133936
d :  0.9838587037818067
k :  975
T :  0.36904761904761907
s0 :  977.2
n :  500


In [ ]:
c1 = binomial_model(s0,k,T,u,d,n,r)[0]
p1 = call_option_parity(s0,c1,k,r,T)
d1,d2,c2= black_scholes(s0,k,r,T,sigma)
p2 = call_option_parity(s0,c2,k,r,T)
print("For Binomial Model with n = ",n)
print("c0 : ",c1)
print("p0 : ",p1)
print("For Black Scholes Model:")
print("c0 : ",c2)
print("p0 : ",p2)
print("As we can see the difference between them is : ",abs(c2-c1))

For Binomial Model with n =  500
c0 :  149.83830865334613
p0 :  129.80104971291985
For Black Scholes Model:
c0 :  149.78560826187748
p0 :  129.74834932145131
As we can see the difference between them is :  0.05270039146864747


In [ ]:
sigma_i = implied_volatility(c2,s0,k,r,T)
print("Sigma Implied is Calculated as : ",sigma_i)

Sigma Implied is Calculated as :  0.5990000000000004


In [ ]:
price = df.iloc[:-180,3]
s,sigma = sigma_from_data(price)
print(s*np.sqrt(252/1))

[-0.012012156448003545, -0.04954573655142988, 0.015748356968139112, -0.03336290366912146, -0.013008313513000734, 0.0401016041685268, 0.049089610196523574, -0.00902261759963753, 0.0428742678306616, 0.0014461318499997134, -0.030816818645987773, 0.01918878065086863, -0.010286644710275426, 0.036262131113536045, 0.08201315166083507, -0.04016604172533476, 0.009517406986890171, 0.0013522652500137541, -0.03436764350420777, 0.005578815027038934, 0.03417967711204518, -0.03278982282299084, 0.01242251999855711, 0.012270092591814401, -0.045746021788203806, -0.015725842305464175, -0.024800687594529493, 0.04618425361985248, -0.034436688269902156, -0.016188727349918282, -0.004460973940625228, -0.010486987495247962, 0.01197619104671562, -0.06612708382270902, -0.01926223279505275, -0.04474648700562273, -0.0309835455199672, 0.030983545519967227, -0.02228027039166556, 0.0017316021642778939, -0.02629424053268704, 0.041745191688406104, -0.010274062976185229, 0.04212922921740313, -0.0016515280384729533, 0.02

In [ ]:
s0 = 977.2
sts = [975,975,975,975,975,975,975,975,975,975]
cts = [38.96,49.8,57,63.5,69.8,73,90,104,117.15,137]
Time = [1,8,12,17,20,24,28,48,67,93]
strategy = self_financing_strategy(s0,sts,cts,r,Time)
for i in range(len(strategy)):
  print("For week ",i+1," the strategy or delta t value is : ",strategy[i])

Implied Volatility for week  1  is  0.9990000000000008
Implied Volatility for week  2  is  0.6910000000000005
Implied Volatility for week  3  is  0.6450000000000005
Implied Volatility for week  4  is  0.6020000000000004
Implied Volatility for week  5  is  0.6100000000000004
Implied Volatility for week  6  is  0.5810000000000004
Implied Volatility for week  7  is  0.6670000000000005
Implied Volatility for week  8  is  0.5820000000000004
Implied Volatility for week  9  is  0.5510000000000004
Implied Volatility for week  10  is  0.5430000000000004
For week  1  the strategy or delta t value is :  0.5280755925128532
For week  2  the strategy or delta t value is :  0.5369521617909147
For week  3  the strategy or delta t value is :  0.5411393868749876
For week  4  the strategy or delta t value is :  0.5454469935695029
For week  5  the strategy or delta t value is :  0.5486024183187258
For week  6  the strategy or delta t value is :  0.5512336687719799
For week  7  the strategy or delta t valu

In [ ]:
s0 = 977.2
sts = [975,975,975,975,975,975,975,975,975,975]
Time = [1,8,12,17,20,24,28,48,67,93]
strategy = self_financing_strategy_sigma(s0,sts,cts,r,Time,s)
for i in range(len(strategy)):
  print("For week ",i+1," the strategy or delta t value is : ",strategy[i])

Implied Volatility for week  1  is  0.4322344719126786
Implied Volatility for week  2  is  0.1528179630760207
Implied Volatility for week  3  is  0.12477534435591038
Implied Volatility for week  4  is  0.10483225780759081
Implied Volatility for week  5  is  0.0966505661415473
Implied Volatility for week  6  is  0.08822949211895083
Implied Volatility for week  7  is  0.08168463719644861
Implied Volatility for week  8  is  0.06238767217795519
Implied Volatility for week  9  is  0.05280584526521296
Implied Volatility for week  10  is  0.04482062681707625
For week  1  the strategy or delta t value is :  0.5412875898486015
For week  2  the strategy or delta t value is :  0.5614660654997686
For week  3  the strategy or delta t value is :  0.5729286967237132
For week  4  the strategy or delta t value is :  0.5871697542989265
For week  5  the strategy or delta t value is :  0.5956608961667025
For week  6  the strategy or delta t value is :  0.6069120356395602
For week  7  the strategy or delta